In [1]:
import cirq
import time
import numpy as np
import random

In [2]:
'''
Takes a qubitString as input and returns a matrix 
that represents the qubit in the standard basis.
Ex: |00> = [1,0,0,0]
'''
def getQubitVector(bitString):
    bitDict = {}
    bitDict['0'] = np.array([1 , 0])
    bitDict['1'] = np.array([0 , 1])
    matrix = [1]
    
    for c in bitString:
        matrix = np.kron(matrix,bitDict[c])
    return matrix

'''
Given a bitstring, return the decimal number 
represented by the bitstring
'''
def getDecimalNo(bitString):
    val = 0
    n = len(bitString)
    for i in range(0,n):
        val = val + (2**i)*(int)(bitString[-i-1])
    return val

'''
Given a value of n, returns all possible bit-strings of size n.
This function will return a list of 2^n bit strings
'''
def getAllPossibleNBitStrings(n):
    if n==1:
        return ['0','1']
    
    children = getAllPossibleNBitStrings(n-1)
    result = []
    for i in children:
        result.append('0'+i)
        result.append('1'+i)
    return result

In [3]:

class FunctionObject:
    def __init__(self,fx,n):
        self.__fx = fx
        self.__n = n
        self.__Uf = self.__createUf()
    
    '''
    Apply the function on the given input.
    '''
    def applyFx(self,input):
        return self.__fx(input)
        
    '''
    The N value which signifies the no. of qubits.
    '''
    def getN(self):
        return self.__n
    
    '''
    The Uf matrix which represents the oracle for this function
    '''
    def getUf(self):
        return self.__Uf
        
    '''
    Using pointer to f(x) and n denoting the no. of qubits,
    it returns an oracle matrix Uf of size 2^(n+1)x2^(n+1) which
    can be used in a Deutsch-Jozsa or Bernstein-Vazirani circuit
    '''
    def __createUf(self):
        Uf = np.zeros((2**(2*self.__n),2**(2*self.__n)))  
    
        #this dictionary represents the correspondence between bit combinations and Uf indices
        indices_dict = {}
        counter = 0

        inputs = getAllPossibleNBitStrings(self.__n*2)
        for i in inputs:
            #input to the function is the first n bits of the elements (bit patterns) from the dictionary
            x = i[0:self.__n]

            #fx represents the output of function f given the input x
            fx = str(self.applyFx(x))

            #b is the last n bits of the bit patterns from the dictionary
            b = i[self.__n:]
            #below we have the (f(x) + b) mod 2
            bfx = bin(int(b,2)+int(fx,2))[2:]

            if(len(bfx)< self.__n):
                bfx = bfx.zfill(self.__n)
            if(len(bfx)>self.__n):
                bfx = bfx[-1*self.__n:]
            
            #the final bit string is the concatenation of the input x and bfx
            result = x + bfx

            #using indices_dict we can now find the index that corresponds to this output
            column = getDecimalNo(result)
            row = getDecimalNo(i)

            #now using the target indiex we can create a bit pattern with all 0s and 1 at the target index position
            Uf[row][column] = 1   
        return Uf

In [4]:
class SimonsFunction(FunctionObject):
    
    '''
    Initializes a function object with the passed in fx and ftype values.
    If no fx and ftype values are passed, 
    it will create the function discussed in Waltrous' notes
    '''
    def __init__(self,n,  fntype=None, fx=None, s=None):
        if fx is None:
            functionObj = self.createDefaultFn(n,fntype)            
            fx = functionObj.fx
            self.s = functionObj.getS()
        else:
            self.s = s            

        FunctionObject.__init__(self, fx, n)        
        
    '''
    Creates our defaul function (Waltrous Notes)
    '''    
    def createDefaultFn(self,n,fntype):
        return DefaultFn(n,fntype)
    
    def getS(self):
        return self.s
    

class DefaultFn(SimonsFunction):
    
    def __init__(self,n, fntype):
        if(n==4):
            self.fn_dict = {'0000':'1010','0001':'1010',
                            '0010':'1100','0011':'1100',
                            '0100':'0000','0101':'0000',
                            '0110':'0101','0111':'0101', 
                            '1000':'0010','1001':'0010',
                            '1010':'1101','1011':'1101',
                            '1100':'1111','1101':'1111',
                            '1110':'0001','1111':'0001'}
            self.s = '0001'
        if(n==3 and fntype==None): #110
            self.fn_dict = {'000':'101','001':'010','010':'000','011':'110','100':'000','101':'110','110':'101','111':'010'}
            self.s = '110'
        if(n==3 and fntype==1): #001
            self.fn_dict = {'000':'101','001':'101','010':'000','011':'000','100':'110','101':'110','110':'010','111':'010'}
            self.s = '001'
        if(n==3 and fntype==2): #010
            self.fn_dict = {'000':'101','001':'010','010':'101','011':'010','100':'000','101':'110','110':'000','111':'110'}
            self.s = '010'
        if(n==3 and fntype==3): #011
            self.fn_dict = {'000':'101','011':'101','001':'111','010':'111','100':'000','111':'000', '101': '011','110':'011' }
            self.s = '011'
        if(n==2):
            self.fn_dict = {'00':'10','01':'10','10':'01','11':'01'}  
            self.s = '01'
        
    def fx(self,inputString):
        return self.fn_dict[inputString]

In [5]:
class Oracle(cirq.Gate):
    def __init__(self, n, UfMatrix, name):
        self.__n=n
        self.__UfMatrix=UfMatrix
        self.__name=name
    def num_qubits(self):
        return self.__n 
    def _unitary_(self):
        return np.squeeze(np.asarray(self.__UfMatrix))
        
    def __str__(self):
        return self.__name

In [6]:
def runMainCircuit(functionObj,debug=False):
    #create a matrix representing Uf
    UfMatrix = functionObj.getUf()
    
    #for a n bit function, we need n+1 qubits(one ancilla bit)
    n = functionObj.getN()*2
    
    # creating an instance of Uf    
    c=cirq.Circuit()
    
    
    qubits = cirq.LineQubit.range(n)
    
    # adding Hadamard gates to all qubits
    for i in range(0,int(n/2)):
        c.append([cirq.H(qubits[i])])
    
    
    # creating Uf gate
    uf_sim= Oracle(n, UfMatrix, "UF_Simon")
    
    # adding Uf gate
    c.append(uf_sim(*qubits))
    
    # helper bit does not require H gate. Result is treated as trash/ garbage    
    for i in range(0,int(n/2)):
        c.append([cirq.H(qubits[i])])        
    
    # measurements
    for i in range(0,int(n/2)):
        c.append(cirq.measure(qubits[i])) 
       
    if debug:
        print(c)
    
    simulator = cirq.Simulator()
    
    result = simulator.run(c, repetitions=1)
    
    if debug:
        print(result)
    
    return result

In [7]:
def allZeros(lst):
    allzeros = True
    for l in lst:
        if l!=0:
            allzeros = False
            break
    return allzeros

def xorLists(lst1,lst2):
    result = []
    for l1,l2 in zip(lst1,lst2):
        if l1==l2:
            result.append(0)
        else:
            result.append(1)
    return result

def addRow(vectors, z,columnToMSBOneDict):    
    while allZeros(z) is False:    
        msbOne = z.index(1)
        if msbOne in columnToMSBOneDict.keys():
            otherRow = columnToMSBOneDict[msbOne]
            z = xorLists(vectors[otherRow],z)
        else:
            break
                
    if(allZeros(z)):
        return [],-1
    else:
        msbOne = z.index(1)
        keysList = list(columnToMSBOneDict.keys())
        i = 0
        for i in range(len(keysList)):
            if keysList[i] > msbOne:
                break
            i+=1
        return z,i            
    
def getResultsFromDict(resultsDict,n):
    results = []
    for i in range(n):
        if resultsDict.measurements[str(i)][0][0] == True:
            results.append(1)
        else:
            results.append(0)

    return results

def addNthVector(vectors,debug=False):
    if debug:
        print("Add nth vector")
        print(vectors)
    newRow = [0 for p in range(len(vectors[0]))]
    lastOne = 0
    for i in range(len(vectors)):
        if vectors[i][i]==1:
            lastOne = i
        else:
            break
            
    newRow[lastOne+1]=1
    vectors.insert(lastOne+1,newRow)
    if debug:
        print(vectors)
    return newRow, vectors


def getS(functionObj,debug=False):
    columnToMSBOneDict = {}
    vectors = []    
    start_time = time.time()
    callsToQuantumCircuit = 0
    while len(vectors) != functionObj.getN()-1 :
        callsToQuantumCircuit+=1
        results = getResultsFromDict(runMainCircuit(functionObj),functionObj.getN())
        if debug:
            print(results)
        z,row = addRow(vectors, results,columnToMSBOneDict)
        if debug:
            print("z","row")
            print(z,row)
        if row is not -1:
            vectors.insert(row,z)
            columnToMSBOneDict[z.index(1)]=row  
    end_time = time.time()
    timeTaken = end_time - start_time
    
    newRow,newVectors = addNthVector(vectors)
    
    if debug:
        print("N Vectors")
        print(newVectors)
        
    a = np.array(newVectors)
    b = np.zeros(functionObj.getN())
    b[newRow.index(1)] = 1
    y = np.linalg.solve(a,b)
    y = [i%2 for i in y]
    x = ''
    for i in range(len(y)):
        x+=str(int(y[i]))
        
    return x,callsToQuantumCircuit,timeTaken

In [8]:
'''
Driver that verifies the validity of the quantum circuits for different F configurations.
'''
class SimonsDriver():
    def __init__(self,n,fnType=None):
        if fnType is None:
            self.functionObj = SimonsFunction(n)
        else:
            self.functionObj = SimonsFunction(n,fnType)
        
    def runAndVerifyQuantumCircuit(self,debug=False):
        print("*"*20)
        x,callsToQuantumCircuit,circuitRunTime = getS(self.functionObj)
        if x is None:
            return
        
        assert x == self.functionObj.getS()
        print("N : {} , QuantumCircuitCalls : {}, S: {}, Time : {}".format(self.functionObj.getN(),callsToQuantumCircuit, x, circuitRunTime))
        print("*"*20)

# Simulation 1

In [18]:
SimonsDriver(2).runAndVerifyQuantumCircuit()
SimonsDriver(3).runAndVerifyQuantumCircuit()
SimonsDriver(3,2).runAndVerifyQuantumCircuit()
SimonsDriver(3,2).runAndVerifyQuantumCircuit()

********************
N : 2 , QuantumCircuitCalls : 1, S: 01, Time : 0.002541065216064453
********************
********************
N : 3 , QuantumCircuitCalls : 3, S: 110, Time : 0.00923609733581543
********************
********************
N : 3 , QuantumCircuitCalls : 2, S: 010, Time : 0.006392955780029297
********************
********************
N : 3 , QuantumCircuitCalls : 12, S: 010, Time : 0.030536890029907227
********************


# Simulation 2

In [19]:
SimonsDriver(2).runAndVerifyQuantumCircuit()
SimonsDriver(3).runAndVerifyQuantumCircuit()
SimonsDriver(3,2).runAndVerifyQuantumCircuit()
SimonsDriver(3,2).runAndVerifyQuantumCircuit()

********************
N : 2 , QuantumCircuitCalls : 7, S: 01, Time : 0.014743804931640625
********************
********************
N : 3 , QuantumCircuitCalls : 4, S: 110, Time : 0.010405302047729492
********************
********************
N : 3 , QuantumCircuitCalls : 5, S: 010, Time : 0.016025066375732422
********************
********************
N : 3 , QuantumCircuitCalls : 3, S: 010, Time : 0.006612062454223633
********************


# Simulation 3

In [9]:
SimonsDriver(2).runAndVerifyQuantumCircuit()
SimonsDriver(3).runAndVerifyQuantumCircuit()
SimonsDriver(3,2).runAndVerifyQuantumCircuit()
SimonsDriver(3,2).runAndVerifyQuantumCircuit()

********************
N : 2 , QuantumCircuitCalls : 2, S: 01, Time : 0.021193981170654297
********************
********************
N : 3 , QuantumCircuitCalls : 2, S: 110, Time : 0.007719278335571289
********************
********************
N : 3 , QuantumCircuitCalls : 2, S: 010, Time : 0.006408214569091797
********************
********************
N : 3 , QuantumCircuitCalls : 3, S: 010, Time : 0.008108139038085938
********************
